###  loading dataset from drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls /content/drive/MyDrive/MSD/

license.txt		Task03_Liver.tar	Task06_Lung.tar		  Task09_Spleen.tar
Task01_BrainTumour.tar	Task04_Hippocampus.tar	Task07_Pancreas.tar	  Task10_Colon.tar
Task02_Heart.tar	Task05_Prostate.tar	Task08_HepaticVessel.tar


### cloning github repo

In [2]:
!git clone https://github.com/arrafi-musabbir/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation.git

Cloning into '3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation'...
remote: Enumerating objects: 219, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 219 (delta 55), reused 58 (delta 25), pack-reused 108
Receiving objects: 100% (219/219), 2.47 MiB | 771.00 KiB/s, done.
Resolving deltas: 100% (80/80), done.


In [12]:
!pip install simpleitk
!pip install tensorboardx
!pip install -q batchgenerators==0.20.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 10.3 MB/s eta 0:00:00


### create `dataset` directory

In [3]:
!mkdir /content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation/dataset

### extract `.tar` files to dataset directory

In [19]:
!tar -xf /content/drive/MyDrive/MSD/Task02_Heart.tar -C /content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation/dataset

In [20]:
!tar -xf /content/drive/MyDrive/MSD/Task04_Hippocampus.tar -C /content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation/dataset

### extracted datasets

In [10]:
!ls /content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation/dataset

Task02_Heart  Task04_Hippocampus


In [21]:
## remove unwanted files
%cd /content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation/dataset
!find  . -name '.*' -exec rm {} \;
!ls -la  /content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation/dataset/

/content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation/dataset
rm: cannot remove '.': Is a directory


### changing current directory to the cloned directory

In [9]:
%cd '/content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation'

/content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation


### explore extracted datasets

In [10]:
!python data_explore.py -h

usage: data_explore.py [-h] [--task TASK] [--root ROOT]

msd

options:
  -h, --help   show this help message and exit
  --task TASK
  --root ROOT  path to save msd_meta.csv


In [24]:
!python data_explore.py

/content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation
task..., patID...
100% 2/2 [00:00<00:00, 435.30it/s]
100% 24/24 [00:00<00:00, 581868.76it/s]
task:   0% 0/2 [00:00<?, ?it/s]
f_path:   0% 0/20 [00:00<?, ?it/s]
f_path:   5% 1/20 [00:00<00:07,  2.52it/s]
f_path:  10% 2/20 [00:00<00:07,  2.34it/s]
f_path:  15% 3/20 [00:01<00:07,  2.30it/s]
f_path:  20% 4/20 [00:01<00:06,  2.48it/s]
f_path:  25% 5/20 [00:02<00:06,  2.37it/s]
f_path:  30% 6/20 [00:02<00:05,  2.44it/s]
f_path:  35% 7/20 [00:02<00:04,  2.72it/s]
f_path:  40% 8/20 [00:03<00:04,  2.90it/s]
f_path:  45% 9/20 [00:03<00:03,  3.02it/s]
f_path:  50% 10/20 [00:03<00:03,  3.20it/s]
f_path:  55% 11/20 [00:03<00:02,  3.13it/s]
f_path:  60% 12/20 [00:04<00:02,  3.25it/s]
f_path:  65% 13/20 [00:04<00:02,  3.33it/s]
f_path:  70% 14/20 [00:04<00:01,  3.29it/s]
f_path:  75% 15/20 [00:05<00:01,  3.67it/s]
f_path:  80% 16/20 [00:05<00:01,  3.72it/s]
f_path:  85% 17/20 [00:05<00:00,  3.55it/s]
f_path:  90% 18/20 [00:05<00:

### preprocess datasets

In [25]:
# using maximum number of cores available
import multiprocessing
available_cores = multiprocessing.cpu_count()

!python preprocess_taskSep.py  --nProc available_cores --tasks Task02_Heart Task04_Hippocampus

  0% 0/2 [00:00<?, ?it/s]Task02_Heart
file path is :/content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation/dataset/Task02_Heart/dataset.json
ids[0]:la_003.nii.gz, current time:20231222_1146
ids[0]:la_001.nii.gz, current time:20231222_1146
NOT fusing cancer to organ!
 50% 1/2 [02:45<02:45, 165.03s/it]Task04_Hippocampus
file path is :/content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation/dataset/Task04_Hippocampus/dataset.json
ids[0]:hippocampus_001.nii.gz, current time:20231222_1148
ids[0]:hippocampus_002.nii.gz, current time:20231222_1148
NOT fusing cancer to organ!
100% 2/2 [03:09<00:00, 94.70s/it]


### train the model with adapters

In [5]:
%cd /content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation

/content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation


In [6]:
## to see available arguments
!python train_model_wt_adapters.py -h

usage: train_model_wt_adapters.py [-h] [--tasks TASKS [TASKS ...]] [--trainMode TRAINMODE]
                                  [--module MODULE] [--ckp CKP] [--resume_ckp RESUME_CKP]
                                  [--resume_epoch RESUME_EPOCH] [--fold FOLD] [--model MODEL]
                                  [--out_tag OUT_TAG] [--base_outChans BASE_OUTCHANS]
                                  [--predict PREDICT] [--pred_epoch PRED_EPOCH]

u2net

options:
  -h, --help            show this help message and exit
  --tasks TASKS [TASKS ...]
                        Task(s) to be trained
  --trainMode TRAINMODE
                        Task adaptation mode
  --module MODULE       specific module type: series_adapter, parallel_adapter, separable_adapter
  --ckp CKP             dir to load ckp for transfer learning
  --resume_ckp RESUME_CKP
                        dir to load ckp for evaluation or training
  --resume_epoch RESUME_EPOCH
                        epoch of resume_ckp
  --fold FOLD   

In [9]:
!python train_model_wt_adapters.py --module parallel_adapter --tasks Task02_Heart Task04_Hippocampus

file path is :/content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation/dataset/Task02_Heart/dataset.json
file path is :/content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation/dataset/Task04_Hippocampus/dataset.json
[1222 12:15:20 @logger.py:135] Directory '/content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation/results/res_u2net3d_universal/Task02_Heart_Task04_Hippocampus/train_log/logger' backuped to '/content/3D-Universal-U-Net-for-Multi-Domain-Medical-Image-Segmentation/results/res_u2net3d_universal/Task02_Heart_Task04_Hippocampus/train_log/logger1222-121520'
[1222 12:15:20 @logger.py:90] Argv: train_model_wt_adapters.py --module parallel_adapter --tasks Task02_Heart Task04_Hippocampus
[1222 12:15:20 @train_model_wt_adapters.py:95] --------------------------------Training for universal: Task02_Heart_Task04_Hippocampus--------------------------------
[1222 12:15:20 @u2net3d.py:298] ------- base_outChans is 16
[1222 12:15:21 @train_model_